In [ ]:
import cv2 as cv
from skimage.measure import  regionprops_table
import numpy as np
import os
import pandas as pd
from PIL import Image

In [ ]:
# from Dieter Baumgartner
# labels the segmented images from Ilastik

def watershed_seeded(Ilastik):
    # Ilastik = Ilastik[300:500, 100:300]
    center = np.where(Ilastik == 1, 255, 0)
    center = np.uint8(center)

    seg = np.where(Ilastik <= 2, 255, 0)
    seg = np.uint8(seg)

    unknown = cv.subtract(seg, center)
    markers = cv.connectedComponents(center)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers[1]+1

    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0

    img2 = np.stack((seg,)*3, axis=2)
    img2 = cv.normalize(img2, None, 255, 0, cv.NORM_MINMAX, cv.CV_8U)   

    # return markers
    return cv.watershed(img2,markers)

In [ ]:
# take labeled segmented images and count cells
def number_of_cells(labeled):

    number_of_cells = []

    for n in range(1,len(labeled)+1):
        props = regionprops_table(labeled[n-1], 
                          properties=['area', 
                                      'equivalent_diameter',
                                      'centroid'])

        df = pd.DataFrame(props)
        cell_area = df["area"].median()
        df_cut = pd.cut(df.area, bins=[0,cell_area, 2*cell_area, 3*cell_area]) 
        df_counts = df_cut.value_counts(sort=False)
        counts =  df_cut.value_counts().tolist()
        number_of_cells.append(counts[0]+counts[1]*2+counts[2]*3) #approximately counts aggregated cells

    return number_of_cells

In [ ]:
# reads 20 images in the mcherry channel segmented by Ilastik and label foreground that contains cells
Ilastik = []
labeled_mcherry = []

for n in range(1,21):
    fname = f"example_images/Control Sample 10xdil Image{n:03}0001_seg.tif"
    Ilastik.append(np.array(Image.open(fname)))
    labeled_mcherry.append(watershed_seeded(Ilastik[n-1]))

In [ ]:
# count cells in the 20 images from mcherry channel

n_ctrl_mcherry = number_of_cells(labeled_mcherry)
print(n_ctrl_mcherry)

[1407, 1284, 1367, 1440, 1413, 1467, 1467, 1526, 1425, 1372, 1216, 1284, 1166, 1271, 1268, 1467, 1370, 1700, 1847, 1486]
